In [1]:
import os
import io
import sys
import json
import boto3
import argparse
import datetime as dt
import pandas as pd
import numpy as np

In [2]:
stream_name = 'customer_churn_stream'
region = 'us-east-1'
datafile = 'churn.txt'

In [3]:
# Generates a list of records to be sent to kinesis
def generate_kinesis_record(datafile):
    df = pd.read_csv(datafile, header='infer')
    records = json.loads(df.to_json(orient='records'))
    return records

In [11]:
session = boto3.Session(profile_name='684423739646')
kinesis_client = session.client('kinesis', region)

In [15]:
for row in generate_kinesis_record(datafile):
    data = json.dumps(row)
    stream_record = [{'Data': bytes(data, 'utf-8'), 'PartitionKey': 'partition_key'}]
    print(stream_record)
    # kinesis_client.put_records(StreamName=stream_name, Records=stream_record)

[{'Data': b'{"State": "KS", "Account Length": 128, "Area Code": 415, "Phone": "382-4657", "Int\'l Plan": "no", "VMail Plan": "yes", "VMail Message": 25, "Day Mins": 265.1, "Day Calls": 110, "Day Charge": 45.07, "Eve Mins": 197.4, "Eve Calls": 99, "Eve Charge": 16.78, "Night Mins": 244.7, "Night Calls": 91, "Night Charge": 11.01, "Intl Mins": 10.0, "Intl Calls": 3, "Intl Charge": 2.7, "CustServ Calls": 1, "Churn?": "False."}', 'PartitionKey': 'partition_key'}]
[{'Data': b'{"State": "OH", "Account Length": 107, "Area Code": 415, "Phone": "371-7191", "Int\'l Plan": "no", "VMail Plan": "yes", "VMail Message": 26, "Day Mins": 161.6, "Day Calls": 123, "Day Charge": 27.47, "Eve Mins": 195.5, "Eve Calls": 103, "Eve Charge": 16.62, "Night Mins": 254.4, "Night Calls": 103, "Night Charge": 11.45, "Intl Mins": 13.7, "Intl Calls": 3, "Intl Charge": 3.7, "CustServ Calls": 1, "Churn?": "False."}', 'PartitionKey': 'partition_key'}]
[{'Data': b'{"State": "NJ", "Account Length": 137, "Area Code": 415, "

[{'Data': b'{"State": "ID", "Account Length": 114, "Area Code": 415, "Phone": "381-2376", "Int\'l Plan": "no", "VMail Plan": "no", "VMail Message": 0, "Day Mins": 202.1, "Day Calls": 100, "Day Charge": 34.36, "Eve Mins": 195.7, "Eve Calls": 102, "Eve Charge": 16.63, "Night Mins": 291.8, "Night Calls": 120, "Night Charge": 13.13, "Intl Mins": 13.3, "Intl Calls": 5, "Intl Charge": 3.59, "CustServ Calls": 2, "Churn?": "False."}', 'PartitionKey': 'partition_key'}]
[{'Data': b'{"State": "PA", "Account Length": 141, "Area Code": 510, "Phone": "365-8114", "Int\'l Plan": "no", "VMail Plan": "no", "VMail Message": 0, "Day Mins": 215.6, "Day Calls": 113, "Day Charge": 36.65, "Eve Mins": 200.6, "Eve Calls": 81, "Eve Charge": 17.05, "Night Mins": 153.8, "Night Calls": 107, "Night Charge": 6.92, "Intl Mins": 12.4, "Intl Calls": 6, "Intl Charge": 3.35, "CustServ Calls": 1, "Churn?": "False."}', 'PartitionKey': 'partition_key'}]
[{'Data': b'{"State": "NM", "Account Length": 132, "Area Code": 408, "Ph

[{'Data': b'{"State": "TN", "Account Length": 148, "Area Code": 415, "Phone": "419-5501", "Int\'l Plan": "no", "VMail Plan": "yes", "VMail Message": 36, "Day Mins": 77.6, "Day Calls": 141, "Day Charge": 13.19, "Eve Mins": 207.0, "Eve Calls": 60, "Eve Charge": 17.6, "Night Mins": 255.7, "Night Calls": 115, "Night Charge": 11.51, "Intl Mins": 10.9, "Intl Calls": 2, "Intl Charge": 2.94, "CustServ Calls": 1, "Churn?": "False."}', 'PartitionKey': 'partition_key'}]
[{'Data': b'{"State": "CT", "Account Length": 93, "Area Code": 415, "Phone": "404-4809", "Int\'l Plan": "no", "VMail Plan": "no", "VMail Message": 0, "Day Mins": 271.1, "Day Calls": 101, "Day Charge": 46.09, "Eve Mins": 237.4, "Eve Calls": 133, "Eve Charge": 20.18, "Night Mins": 145.4, "Night Calls": 103, "Night Charge": 6.54, "Intl Mins": 8.4, "Intl Calls": 6, "Intl Charge": 2.27, "CustServ Calls": 1, "Churn?": "True."}', 'PartitionKey': 'partition_key'}]
[{'Data': b'{"State": "AZ", "Account Length": 138, "Area Code": 415, "Phone

In [17]:
response = kinesis_client.describe_stream(StreamName=stream_name)
response

{'StreamDescription': {'StreamName': 'customer_churn_stream',
  'StreamARN': 'arn:aws:kinesis:us-east-1:684423739646:stream/customer_churn_stream',
  'StreamStatus': 'ACTIVE',
  'Shards': [{'ShardId': 'shardId-000000000000',
    'HashKeyRange': {'StartingHashKey': '0',
     'EndingHashKey': '340282366920938463463374607431768211455'},
    'SequenceNumberRange': {'StartingSequenceNumber': '49620749161031217401286417052783060790410671426046525442'}}],
  'HasMoreShards': False,
  'RetentionPeriodHours': 24,
  'StreamCreationTimestamp': datetime.datetime(2021, 8, 3, 11, 48, 47, tzinfo=tzlocal()),
  'EnhancedMonitoring': [{'ShardLevelMetrics': []}],
  'EncryptionType': 'NONE'},
 'ResponseMetadata': {'RequestId': 'e8669a52-d27e-dae7-b1e5-1bbff0abbb06',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e8669a52-d27e-dae7-b1e5-1bbff0abbb06',
   'x-amz-id-2': 'fuqCHsOnRUJfLhCFoOKeVmrfX+xUAadkrcbZGlATorSmbs7TYLT+Nhy8jzZbFpb2ZzAiiyYIdC6SCCBdJ7fjCVpX1DgWecck',
   'date': 'Tue, 03 Aug 

In [22]:
shard_id = response['StreamDescription']['Shards'][0]['ShardId']
shard_iterator = kinesis_client.get_shard_iterator(StreamName=stream_name,
                                                      ShardId=shard_id,
                                                      ShardIteratorType='TRIM_HORIZON')
shard_iterator

{'ShardIterator': 'AAAAAAAAAAGI8XLIyhHTZFLwu3/6en4UYnT1vTQT9uJ/8E9V5oJ0zTrenrSjg7o93GtWsidFoN+6qzZM+9QYWx3uAE+P/UltxrqF02inB60FPRzibzL3IcUzp+lJ7uJGAQ5loVmQP/Nr81nIROfIyShKqCW1DPkcnOvlypssPD4HHePT0xyE+/oNPbU8zYJQGHbIVGED38KvYHyXsDJXSJN6p7eVOgr9I/eYR4nEHeyNTay22tXl2IDU/OH14EEB7awGCFB1cKs=',
 'ResponseMetadata': {'RequestId': 'ffa9bf67-d29f-e876-a62a-3c76ae033499',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ffa9bf67-d29f-e876-a62a-3c76ae033499',
   'x-amz-id-2': 'yFljbm6H7xWuegCoL0zXR/yvgIiO311BhkFZUeFyTeaEA2EHVFK8urcnNaiQJuaKy6trJHsRpLoQovgrhgT6zgaYD51xfJB2',
   'date': 'Tue, 03 Aug 2021 17:45:53 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '288'},
  'RetryAttempts': 0}}

In [24]:
my_shard_iterator = shard_iterator['ShardIterator']

record_response = kinesis_client.get_records(ShardIterator=my_shard_iterator, Limit=2)
record_response['Records']

[{'SequenceNumber': '49620749161031217401286417081495049006258225582151041026',
  'ApproximateArrivalTimestamp': datetime.datetime(2021, 8, 3, 12, 15, 44, 559000, tzinfo=tzlocal()),
  'Data': b'{"State": "KS", "Account Length": 128, "Area Code": 415, "Phone": "382-4657", "Int\'l Plan": "no", "VMail Plan": "yes", "VMail Message": 25, "Day Mins": 265.1, "Day Calls": 110, "Day Charge": 45.07, "Eve Mins": 197.4, "Eve Calls": 99, "Eve Charge": 16.78, "Night Mins": 244.7, "Night Calls": 91, "Night Charge": 11.01, "Intl Mins": 10.0, "Intl Calls": 3, "Intl Charge": 2.7, "CustServ Calls": 1, "Churn?": "False."}',
  'PartitionKey': 'partition_key'},
 {'SequenceNumber': '49620749161031217401286417081496257932077840211325747202',
  'ApproximateArrivalTimestamp': datetime.datetime(2021, 8, 3, 12, 15, 44, 615000, tzinfo=tzlocal()),
  'Data': b'{"State": "OH", "Account Length": 107, "Area Code": 415, "Phone": "371-7191", "Int\'l Plan": "no", "VMail Plan": "yes", "VMail Message": 26, "Day Mins": 161.6